In [1]:
import os
os.chdir("..")

In [2]:
import openai
client = openai.Client()

In [3]:
from datasets import load_dataset
blocksworld_type = "big"

dataset = load_dataset(f"dmitriihook/deepseek-r1-qwen-32b-planning-{blocksworld_type}")["train"]

In [5]:
text = dataset[0]["generation"]

In [21]:
steps = text.split("\n\n")

for step in steps:
    print(step)
    print("==="*3)

<think>
Okay, so I have this problem where I need to rearrange some blocks. Let me try to figure out the steps I need to take. 
First, let me understand the initial setup. The blocks are A, B, C, D. The initial conditions are:
- Block A is clear.
- Block B is clear.
- Hand is empty.
- Block A is on top of Block D.
- Block B is on top of Block C.
- Block C is on the table.
- Block D is on the table.
So, visualizing this, I have two separate stacks. One stack is D with A on top, and another stack is C with B on top. Both D and C are on the table. 
My goal is to have:
- Block A on top of Block D.
- Block B on top of Block C.
- Block D on top of Block B.
Wait, that seems a bit confusing. Let me parse that again. The goal is:
- A is on D.
- B is on C.
- D is on B.
So, the final arrangement should be a stack where D is on B, which is on C, and A is on D. So the order from bottom to top would be C, B, D, A. But wait, that can't be because D is on B, which is on C, and A is on D. So the stack 

In [48]:
step_n = 1

print(steps[step_n])

First, let me understand the initial setup. The blocks are A, B, C, D. The initial conditions are:


In [70]:


prompt_template = """Blocksworld is a scenario in which agent needs to stack blocks in a certain way.
The agent has four possible actions: pick up a block, put down a block, stack a block on top of another block, and unstack a block from another block.
The agent can only pick up one block at a time, and can only stack or put down a block if it is holding one.

You will be given a reasoning step from the blocksworld problem solution. 
You have to extract the goal action from the reasoning step. 
Then you have to extract all of the actions that were found to be necessary to reach the goal action in the reasoning step.
If there was not explicitly stated goal action, return null in the goal action field. Then extract all of the actions that were planned in the reasoning step.

If there were no actions considered in the step, return null in the actions field.

Follow the format in the examples below. Answer in the correct JSON format.
You should only use the four possible actions: pick up a block, put down a block, stack a block on top of another block, and unstack a block from another block.

Reasoning step:
1. Unstack A from D.
2. Put down A.
3. Pick up D.
4. Stack D on B.
5. Pick up A.
6. Stack A on D.

Output:
{{"goal_action": null, "actions": [["unstack", "A", "D"], ["put down", "A"], ["pick up", "D"], ["stack", "D", "B"], ["pick up", "A"], ["stack", "A", "D"]]}}


Reasoning step:
1. Move A off of D. Since A is clear, I can unstack A from D. Then put A down somewhere, maybe on the table.

Outpit:
{{"goal_action": ["unstack", "A", "D"], "actions": [["unstack", "A", "D"], ["put down", "A"]]}}

Reasoning step:
{step}

Output:
"""

In [71]:
from pydantic import BaseModel

class ExtractedActions(BaseModel):
    goal_action: list[str] | None
    actions: list[list[str]] | None


In [72]:
from tqdm.auto import tqdm

labeled_examples = []


def check_step(step):
    keywords = ["pick up", "put down", "stack", "unstack"]
    for keyword in keywords:
        if keyword in step.lower():
            return True

for item in tqdm(dataset.select(range(1))):
    text = item["generation"]
    steps = text.split("\n\n")

    group = []

    for step in steps:
        prompt = prompt_template.format(step=step)

        if check_step(step):
            response = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": prompt},
                ],
                max_tokens=200,
                response_format=ExtractedActions
            )

            label = response.choices[0].message.parsed
        else:
            label = None

        group.append({
            "text": step,
            "label": label
        })

    labeled_examples.append(group)

  0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
for x in labeled_examples[0]:

    print(x["text"])
    print("***"*3)
    print(x["label"])
    print("==="*3)

<think>
Okay, so I have this problem where I need to rearrange some blocks. Let me try to figure out the steps I need to take. 
*********
None
First, let me understand the initial setup. The blocks are A, B, C, D. The initial conditions are:
*********
None
- Block A is clear.
- Block B is clear.
- Hand is empty.
- Block A is on top of Block D.
- Block B is on top of Block C.
- Block C is on the table.
- Block D is on the table.
*********
None
So, visualizing this, I have two separate stacks. One stack is D with A on top, and another stack is C with B on top. Both D and C are on the table. 
*********
goal_action=None actions=None
My goal is to have:
*********
None
- Block A on top of Block D.
- Block B on top of Block C.
- Block D on top of Block B.
*********
None
Wait, that seems a bit confusing. Let me parse that again. The goal is:
*********
None
- A is on D.
- B is on C.
- D is on B.
*********
None
So, the final arrangement should be a stack where D is on B, which is on C, and A is 